# Can you solve the human cannonball riddle? - Alex Rosenthal
## [Source](https://www.youtube.com/watch?v=o4MpBV4F3qs&list=WL)

<img src='image/Capture.png' width='500px'>

### [Glitched Failure Video](https://youtu.be/mQMCb4aVaSw)

## Approach

1. Generate all possible Cannon states
1. Create Cannon ruleset
1. Iterate over all possible Cannon states
1. Analyze all states that pass ruleset

## Code I wish I had

```python
ruleset = [rule_1, rule_2, rule_3]
cannon_gen = create_cannon_state_generator()

solutions = []
for cannon_state in cannon_gen:
    passes = True
    for rule in ruleset:
        passes &= rule(cannon_state)
        if not passes:
            break

    if passes:
        solutions.append(cannon_state)
```

### `create_cannon_state_generator`

Generators can be used to make efficient use of space

In [31]:
from itertools import product

list(product(product([1,2,3], repeat=8), repeat=2))

[((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 1, 1, 1)),
 ((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 1, 1, 2)),
 ((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 1, 1, 3)),
 ((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 1, 2, 1)),
 ((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 1, 2, 2)),
 ((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 1, 2, 3)),
 ((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 1, 3, 1)),
 ((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 1, 3, 2)),
 ((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 1, 3, 3)),
 ((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 2, 1, 1)),
 ((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 2, 1, 2)),
 ((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 2, 1, 3)),
 ((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 2, 2, 1)),
 ((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 2, 2, 2)),
 ((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 2, 2, 3)),
 ((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 2, 3, 1)),
 ((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 2, 3, 2)),
 ((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 2, 3, 3)),
 ((1, 1, 1

In [32]:
from itertools import product

def create_cannon_state_generator(layers, min_max_range):
    min_val, max_val = min_max_range
    for cannon_state in product(product(range(min_val, max_val+1), repeat=8), repeat=layers):
        yield cannon_state
    return

In [33]:
csg = create_cannon_state_generator(layers = 2, min_max_range = (1,3))

for _ in range(10):
    print(next(csg))

((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 1, 1, 1))
((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 1, 1, 2))
((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 1, 1, 3))
((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 1, 2, 1))
((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 1, 2, 2))
((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 1, 2, 3))
((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 1, 3, 1))
((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 1, 3, 2))
((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 1, 3, 3))
((1, 1, 1, 1, 1, 1, 1, 1), (1, 1, 1, 1, 1, 2, 1, 1))


43,040,160 possible cannon states!

### Convert flat layer list into 3D orientation

Indices for each side


```python
layer = [0, 1, 2, 3, 4, 5, 6, 7]

         north
     | 0 | 1 | 2 |
west | 3 |   | 4 | east
     | 5 | 6 | 7 |
         south

```

In [ ]:
# Create mapping of flat cannon state layers
# contains the indices for each side
layer_map = {
    'north': [2,1,0],
    'south': [5,6,7],
    'west' : [0,3,5],
    'east' : [7,4,2],
}

In [34]:
from colorama import Fore, Style

def vis_cannon(cannon_state):
    layer_1 = [f'{Fore.GREEN}{val}{Style.RESET_ALL}'.strip() for val in cannon_state[0]] # lower
    layer_2 = [f'{Fore.RED}{val}{Style.RESET_ALL}'.strip() for val in cannon_state[1]] # upper

    layer_map = {
        'north': [2,1,0],
        'south': [5,6,7],
        'west' : [0,3,5],
        'east' : [7,4,2],
    }
    
    lower = \
    """
          LOWER
            n
      | {} | {} | {} |
    w | {} |   | {} | e
      | {} | {} | {} |
            s
    """.format(*layer_1)
    
    upper = \
    """
          UPPER
            n
        | {} | {} | {} |
    w | {} |   | {} | e
        | {} | {} | {} |
            s
    """.format(*layer_2)
        
    north = \
    """
        NORTH
        upper
      | {} | {} | {} |
    | {} | {} | {} |
          lower
    """.format(*[layer_2[i] for i in layer_map['north']], *[layer_1[i] for i in layer_map['north']])
       
    south = \
    """
    SOUTH
    upper
    | {} | {} | {} |
    | {} | {} | {} |
      lower
    """.format(*[layer_2[i] for i in layer_map['south']], *[layer_1[i] for i in layer_map['south']])
       
    east = \
    """
  EAST
  upper
      | {} | {} | {} |
      | {} | {} | {} |
    lower
    """.format(*[layer_2[i] for i in layer_map['east']], *[layer_1[i] for i in layer_map['east']])
       
    west = \
    """
WEST
upper
      | {} | {} | {} |
      | {} | {} | {} |
  lower
    """.format(*[layer_2[i] for i in layer_map['west']], *[layer_1[i] for i in layer_map['west']])
       
    # PRINTING #
    fixed_width = 21
    def set_width(string):
        return f"{string:<{fixed_width}}"

    all_sides = [lower, upper, north, south, east, west]
    for combined_str in zip(*[side.split('\n') for side in all_sides]):
        print(''.join([set_width(string) for string in combined_str]))

In [35]:
# Showing lower and upper indices
vis_cannon([range(8), range(8)])

                                                                                                                              
          LOWER                UPPER              NORTH            SOUTH              EAST               WEST                 
            n                    n                upper            upper              upper              upper                
      | 0 | 1 | 2 |        | 0 | 1 | 2 |      | 2 | 1 | 0 |    | 5 | 6 | 7 |      | 7 | 4 | 2 |      | 0 | 3 | 5 |
    w | 3 |   | 4 | e    w | 3 |   | 4 | e    | 2 | 1 | 0 |    | 5 | 6 | 7 |      | 7 | 4 | 2 |      | 0 | 3 | 5 |
      | 5 | 6 | 7 |        | 5 | 6 | 7 |          lower            lower              lower              lower              
            s                    s                                                                                            


### What should a "rule" look like?

- Will be a function corresponding to each given rule
- Will take all Cannon layers as nested "flat" list representation
- Returns `True` if the rule is meet

__Rules:__
1. There are twice as many _energy cells_ in the upper level as in the lower level
2. Each side of the cannon (made of 6 chambers) has 11 _energy cells_

The cannon was originally meant to have $n$ _energy cells_, but Kanoli made it work with $n-3$. What was this number?

BONUS: How do you arrange the _energy cells_?

In [36]:
# RULE 1: There are twice as many energy cells in the upper level as in the lower level
def rule_1(cannon_state):
    layer_1 = cannon_state[0] # lower
    layer_2 = cannon_state[1] # upper
    
    return sum(layer_1) * 2 == sum(layer_2) 

In [37]:
# RULE 2: Each side of the cannon (made of 6 chambers) has 11 energy cells
def rule_2(cannon_state):
    layer_1 = cannon_state[0] # lower
    layer_2 = cannon_state[1] # upper

    layer_map = {
        'north': [2,1,0],
        'south': [5,6,7],
        'west' : [0,3,5],
        'east' : [7,4,2],
    }
    
    for side in layer_map.keys():
        indices = layer_map[side]
        
        total = 0
        for index in indices:
            total += layer_1[index] + layer_2[index]
        
        # stop early
        if total != 11:
            return False
    return True

# Solution

In [38]:
ruleset = [rule_1, rule_2]
cannon_gen = create_cannon_state_generator(layers = 2, min_max_range = (1,3))

solutions = []
for cannon_state in cannon_gen:
    passes = True
    for rule in ruleset:
        passes &= rule(cannon_state)
        if not passes: # stop early
            break

    if passes:
        solutions.append(cannon_state)
        print('Found solution!', cannon_state)
print("DONE!")

Found solution! ((1, 1, 1, 1, 1, 1, 1, 2), (3, 2, 3, 2, 1, 3, 1, 3))
Found solution! ((1, 1, 1, 1, 1, 1, 1, 3), (3, 3, 2, 3, 3, 2, 3, 1))
Found solution! ((1, 1, 1, 1, 1, 1, 2, 2), (2, 3, 3, 3, 3, 3, 2, 1))
Found solution! ((1, 1, 1, 1, 1, 1, 2, 2), (3, 2, 3, 3, 3, 2, 3, 1))
Found solution! ((1, 1, 1, 1, 1, 1, 2, 2), (3, 3, 2, 3, 3, 2, 2, 2))
Found solution! ((1, 1, 1, 1, 1, 1, 3, 1), (2, 3, 3, 3, 3, 3, 1, 2))
Found solution! ((1, 1, 1, 1, 1, 1, 3, 1), (3, 2, 3, 3, 3, 2, 2, 2))
Found solution! ((1, 1, 1, 1, 1, 1, 3, 1), (3, 3, 2, 3, 3, 2, 1, 3))
Found solution! ((1, 1, 1, 1, 1, 2, 1, 1), (3, 2, 3, 1, 2, 3, 1, 3))
Found solution! ((1, 1, 1, 1, 1, 2, 1, 2), (2, 3, 3, 3, 3, 2, 3, 1))
Found solution! ((1, 1, 1, 1, 1, 2, 1, 2), (3, 3, 2, 3, 3, 1, 3, 2))
Found solution! ((1, 1, 1, 1, 1, 2, 2, 1), (2, 3, 3, 3, 3, 2, 2, 2))
Found solution! ((1, 1, 1, 1, 1, 2, 2, 1), (3, 2, 3, 3, 3, 1, 3, 2))
Found solution! ((1, 1, 1, 1, 1, 2, 2, 1), (3, 3, 2, 3, 3, 1, 2, 3))
Found solution! ((1, 1, 1, 1, 1, 3

In [39]:
len(solutions)

118

118 solutions!
Certianly duplicates, due to rotated orientations

Let's look into solutions with $n-3$

In [40]:
unique_solutions = {}
for solution in solutions:
    layer_1 = solution[0] # lower
    layer_2 = solution[1] # upper

    total_n = sum(layer_1) + sum(layer_2)
    if total_n not in unique_solutions:
        unique_solutions[total_n] = []
    unique_solutions[total_n].append(solution)

In [41]:
unique_solutions

{27: [((1, 1, 1, 1, 1, 1, 1, 2), (3, 2, 3, 2, 1, 3, 1, 3)),
  ((1, 1, 1, 1, 1, 2, 1, 1), (3, 2, 3, 1, 2, 3, 1, 3)),
  ((1, 1, 2, 1, 1, 1, 1, 1), (3, 1, 3, 2, 1, 3, 2, 3)),
  ((2, 1, 1, 1, 1, 1, 1, 1), (3, 1, 3, 1, 2, 3, 2, 3))],
 30: [((1, 1, 1, 1, 1, 1, 1, 3), (3, 3, 2, 3, 3, 2, 3, 1)),
  ((1, 1, 1, 1, 1, 1, 2, 2), (2, 3, 3, 3, 3, 3, 2, 1)),
  ((1, 1, 1, 1, 1, 1, 2, 2), (3, 2, 3, 3, 3, 2, 3, 1)),
  ((1, 1, 1, 1, 1, 1, 2, 2), (3, 3, 2, 3, 3, 2, 2, 2)),
  ((1, 1, 1, 1, 1, 1, 3, 1), (2, 3, 3, 3, 3, 3, 1, 2)),
  ((1, 1, 1, 1, 1, 1, 3, 1), (3, 2, 3, 3, 3, 2, 2, 2)),
  ((1, 1, 1, 1, 1, 1, 3, 1), (3, 3, 2, 3, 3, 2, 1, 3)),
  ((1, 1, 1, 1, 1, 2, 1, 2), (2, 3, 3, 3, 3, 2, 3, 1)),
  ((1, 1, 1, 1, 1, 2, 1, 2), (3, 3, 2, 3, 3, 1, 3, 2)),
  ((1, 1, 1, 1, 1, 2, 2, 1), (2, 3, 3, 3, 3, 2, 2, 2)),
  ((1, 1, 1, 1, 1, 2, 2, 1), (3, 2, 3, 3, 3, 1, 3, 2)),
  ((1, 1, 1, 1, 1, 2, 2, 1), (3, 3, 2, 3, 3, 1, 2, 3)),
  ((1, 1, 1, 1, 1, 3, 1, 1), (2, 3, 3, 3, 3, 1, 3, 2)),
  ((1, 1, 1, 1, 2, 1, 1, 2), (2, 3, 3, 

In [42]:
unique_solutions.keys()

dict_keys([27, 30])

We find 2 different kinds of solutions:
1. Solutions that meet the requirments with $n = 27$
1. Solutions that meet the requirments with $n = 30$


In [43]:
unique_solutions[27]

[((1, 1, 1, 1, 1, 1, 1, 2), (3, 2, 3, 2, 1, 3, 1, 3)),
 ((1, 1, 1, 1, 1, 2, 1, 1), (3, 2, 3, 1, 2, 3, 1, 3)),
 ((1, 1, 2, 1, 1, 1, 1, 1), (3, 1, 3, 2, 1, 3, 2, 3)),
 ((2, 1, 1, 1, 1, 1, 1, 1), (3, 1, 3, 1, 2, 3, 2, 3))]

4 solutions have 27 total _energy cells_ (likely duplicates due to rotation)

In [44]:
vis_cannon(unique_solutions[27][0])

                                                                                                                              
          LOWER                UPPER              NORTH            SOUTH              EAST               WEST                 
            n                    n                upper            upper              upper              upper                
      | 1 | 1 | 1 |        | 3 | 2 | 3 |      | 3 | 2 | 3 |    | 3 | 1 | 3 |      | 3 | 1 | 3 |      | 3 | 2 | 3 |
    w | 1 |   | 1 | e    w | 2 |   | 1 | e    | 1 | 1 | 1 |    | 1 | 1 | 2 |      | 2 | 1 | 1 |      | 1 | 1 | 1 |
      | 1 | 1 | 2 |        | 3 | 1 | 3 |          lower            lower              lower              lower              
            s                    s                                                                                            


OBSERVATION:
- The lower level has all 1's except for a single corner 2
- The upper level has 3's on each corner. And 2's in the center, except for the sides around the corner where the lower level has the 2.